In [2]:
!pip install pandas
!pip install numpy

In [3]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [4]:
movies = pd.read_csv('tmdb_5000_movies.csv')

In [5]:
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
movies_df = movies[['id','title','genres','vote_average','vote_count','popularity','keywords','overview']]
print(movies_df)

id                                     title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4798    9367                               El Mariachi   
4799   72766                                 Newlyweds   
4800  231617                 Signed, Sealed, Delivered   
4801  126186                          Shanghai Calling   
4802   25975                         My Date with Drew   

                                                 genres  vote_average  \
0     [{"id": 28, "name": "Action"}, {"id": 12, "nam...           7.2   
1     [{"id": 12, "name": "Adventure"}, {"id": 14, "...           6.9   
2     [{"id": 28, "name": "Action"}, {"id": 12, "nam...           6.3   
3     [{"id": 28, "na

In [7]:
pd.set_option('max_colwidth',100)
#각각의 장르와 그 장르의 id를 추출해 낼 수 있다
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [8]:
#literal_eval() 함수를 사용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다
from ast import literal_eval
movies_df['genres'] = movies_df['genres'].apply(literal_eval) #genres 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체들을 가진다
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)


In [9]:
#genres 칼럼에서 'name' 키에 해당하는 값을 추출하기 위해서 apply lambda 식을 이용한다
#apply(lambda x : [y['name] for y in x])와 같이 변환하면 리스트 내 여러개의 딕셔너리의 'name'에 해당하는 값을 찾아 이를 리스트 객체로 변환한다
movies_df['genres'] = movies_df['genres'].apply(lambda x : [y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [y['name'] for y in x])
movies_df[['genres','keywords'][:1]]


,genres
0,"[Action, Adventure, Fantasy, Science Fiction]"
1,"[Adventure, Fantasy, Action]"
2,"[Action, Adventure, Crime]"
3,"[Action, Crime, Drama, Thriller]"
4,"[Action, Adventure, Science Fiction]"
...,...
4798,"[Action, Crime, Thriller]"
4799,"[Comedy, Romance]"
4800,"[Comedy, Drama, Romance, TV Movie]"
4801,[]


In [10]:
#genres 칼럼은 여러개의 개별장르가 리스트로 구성되어 있다
#만일 영화 A가 genres의 [Action, Adventure,Fantasy, Science Fiction]으로 되어 있고
#영화 B의 genres가 [Adventure, Fantasy, Action]으로 돼 있다면 어떻게 장르별 유사도를 측정할 수 있을까?

#가장 간단한 방법으로는 genres를 문자열로 변경한 뒤 이를 CountVectorizer로 피처 벡터화한 행렬 데이터 값을 코사인 유사도로 비교하는 것

#genres 칼럼을 기반으로 하는 콘텐츠 기반 필터링
#1. 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환합니다
#2. genres 문자열을 피처 벡터화 행렬로 변환한 데이터 세트를 코사인 유사도를 통해 비교합니다. 이를 위해 데이터 세트의 레코드별로 다 레코드와 장르에서 코사인 유사도 값을 가지는 객체를 생성합니다
#3. 장르 유사도가 높은 영화 중에 평점이 높은 순으로 영화를 추천한다


In [11]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x))
count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape) # 4803개의 레코드와 276개의 개별단어 피처로 구성된 피처 벡터 행렬이 만들어졌다
#이렇게 생성된 피처 벡터 행렬에 사이킷런의 cosine_similarity()를 이용해 코사인 유사도를 계산한다

(4803, 276)


In [12]:
#피처 벡터화된 행렬에 cosine_similary()를 적용한 코드
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim)
#cosine_similary로 호출로 생성된 genre_sim 객체는 movies_df의 genre_literal 칼럼을 피처 벡터화한 행렬(genre_mat) 데이터의 행(레코드)별 유사도 정보를 지니고 있다
#결국은 movies_df DataFrame의 행별 장르 유사도 값을 가지고 있는 것이다
#movies_df를 장르기준으로 콘텐츠 기반 필터링을 수행하려면 movies_df의 개별 레코드ㅔ 대해서 가장 장르 유사도가 높은 순으로 다른 레코드를 추출해야 하는데 이를 위해서 genre_sim을 사용한다

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]
 [0.59628479 1.         0.4        ... 0.         0.         0.        ]
 [0.4472136  0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [13]:
#genre_sim 객체의 기준 행별로 비교대산니 되는 행의 유사도 값이 높은 순으로 정렬된 행렬의 위치 인덱스 값을 추출하면 된다
#값이 높은 순으로 정렬된 비교대상 행의 유사도 값이 아니라 비교대상 행의 위치 인덱스임에 주의
#argsort()[:.::-1]을 이용하면 유사도가 높은 순서로 정리된 genre_sim 객체의 비교 행 위치 인덱스 값을 간편하게 얻을 수 있다

#argsort()[:,::-1]를 사용해 높은 순으로 정렬된 비교 행 위치 인덱스를 가져오고 그 중에 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])
#2번째로 출력된 3494가 나를 제외하고 가장 유사도가 높다

[[   0 3494  813 ... 3038 3037 2401]]


In [14]:
#장르 유사도에 따라 영화를 추천하는 함수
#함수명 : find_sim_movie()
#movies_df DataFrame : 기반 데이터
#genre_sim_sorted_ind : 레코드별 장르 코사인 유사도 인덱스
#DataFrame : 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가짐
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    #인자로 입력된 movies)df DataFrame에서 'title' 칼럼이 입력된 title_name 값인 DataFrame 추출
    title_movie = df[df['title']== title_name]

    #title_named를 가진 DataFrame 의 인덱스 객체를 ndarray로 반환
    #sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
    title_index = title_movie.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]

    #추출된 top_n index 출력, top_n index는 2차원 데이터임
    #dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)

    return df.iloc[similar_indexes]

In [15]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind,'The Godfather', 10)
similar_movies[['title','vote_average']]
#좀 더 개선이 필요한 형태의 추천 시스템이 나옴

[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [16]:
#더 많은 후보군을 선정한 뒤 영화의 평점에 따라 필터링을 해서 최종 추천하는 방식으로 변경
#영화의 평점 정보인 vote_average를 이용
#vote_Average는 0부터 10점 만점까지의 점수로 돼 있는데 여러 관객들이 평가한 평점을 평균한 것이다
#하지만 왜곡된 데이터를 가지고 있기 때문에 sort_values()를 이용해 평점의 오름차순으로 10개를 출력해 보겠다

movies_df[['title','vote_average','vote_count']].sort_values('vote_average',ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


In [17]:
#평가된 수가 적은 영화는 진짜 유명한 작품보다 높은 평점을 가질 수 있다 -> 신뢰도가 떨어진다
#유명한 영화 평점 사이트인 IMDB에서는 평가횟수에 대한 가중치가 부여된 평점 방식을 사용한다
#가중평점 = (v/(v+m)) * R + (m/(v+m)) * C
#v : 개별 영화에 평점을 투표한 횟수
#m : 평점을 부여하기 위한 최소 투표 횟수
#R : 개별 영화에 대한 평균 평점
#C : 전체 영화에 대한 평균 평점
C = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('C: ',round(C,3),'m: ',round(m,3))

C:  6.092 m:  370.2


In [18]:
#기존 평점을 새로운 가중 평점으로 변경하는 함수를 생성하고 이를 사용해 새로운 평점 정보인 weighted값을 만든다
percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']

    return  (v/(v+m)) * R + (v/(v+m)) * C


movies['weighted_vote'] = movies.apply(weighted_vote_average,axis=1)

In [19]:
#새로 부여된 weighted_vote 평점이 높은 순으로 상위 10개 추출
movies[['title','vote_average','weighted_vote','vote_count']].sort_values(
    'weighted_vote',ascending=False)[:10] 


,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,13.962213,8205
65,The Dark Knight,8.2,13.864522,12002
662,Fight Club,8.3,13.847566,9413
96,Inception,8.1,13.820137,13752
3232,Pulp Fiction,8.3,13.786595,8428
95,Interstellar,8.1,13.724623,10867
809,Forrest Gump,8.2,13.654491,7927
3337,The Godfather,8.4,13.635580,5893
329,The Lord of the Rings: The Return of the King,8.1,13.569239,8064
262,The Lord of the Rings: The Fellowship of the Ring,8.0,13.517317,8705


In [20]:
#이제 새롭게 정의된 평점 기준에 따라서 영화를 추천한다
#장르 유사성이 높은 영화를 top_n의 2배수만큼 후보군으로 선정한 뒤에 weighted_vote 칼럼이 높은 순으로 top_n만큼 추출하는 방식으로
#find_sim_movie()함수를 변경한다

def find_sim_movie(df, sorted_ind, title_name, top_n =10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values

    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexs = similar_indexes.reshape(-1)

    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    print(similar_indexes)
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote',ascending=False)[:top_n]

similar_movies = find_sim_movie(movies,genre_sim_sorted_ind,'The Godfather',10)
similar_movies[['title','vote_average','weighted_vote']]

[2731 1243 3636 1946 2640 4065 1847 4217  883 3866 3112 4041  588 3378
  281 1663 1464 1149 2839]


,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,12.955361
1847,GoodFellas,8.2,12.779690
883,Catch Me If You Can,7.7,12.566333
3866,City of God,8.1,11.786741
1149,American Hustle,6.8,11.390006
281,American Gangster,7.4,10.824293
1663,Once Upon a Time in America,8.2,10.615850
2839,Rounders,6.9,7.048398
588,Wall Street: Money Never Sleeps,5.8,6.791984
4041,This Is England,7.4,6.679839
